In [1]:
import requests 
import time
from tqdm import tqdm
import json
from multiprocessing import Pool
from fake_useragent import UserAgent

In [2]:
collection = 'CRWEAPONS-e5ab49'
sleep_time = 0.4

In [3]:
url = f'https://api.elrond.com/collections/{collection}'
collection_info = requests.get(url).json()

In [4]:
identifiers = set()

In [5]:
start, stop, step = 0, 10000, 100
for i in tqdm(range(start, stop, step)):
    url = f'https://api.elrond.com/collections/{collection}/nfts?from={i}&size={step}'
    response = requests.get(url, headers={'User-Agent': UserAgent().random})
    response = response.json()
    if len(response) == 0:
        break
    identifiers.update({nft['identifier'] for nft in response})
    time.sleep(sleep_time)
identifiers = list(identifiers)

 22%|██▏       | 22/100 [00:21<01:16,  1.02it/s]


In [6]:
def foo(chunk):
    sub_nfts = {}
    try:
        for identifier in chunk:
            time.sleep(sleep_time)
            url = f'https://api.elrond.com/nfts/{identifier}'
            headers = {'User-Agent': UserAgent().random}
            sub_nfts[identifier] = requests.get(url, headers=headers).json()
    except:
        pass
    return sub_nfts

In [7]:
nfts = {}

In [8]:
while True:
    to_compute = [identifier for identifier in identifiers if identifier not in nfts]
    for identifier in tqdm(to_compute):
        try:
            time.sleep(sleep_time)
            url = f'https://api.elrond.com/nfts/{identifier}'
            headers = {'User-Agent': UserAgent().random}
            nfts[identifier] = requests.get(url, headers=headers).json()
        except:
            pass
    if len(nfts) == len(identifiers):
        break
    time.sleep(sleep_time*40)

100%|██████████| 2138/2138 [35:26<00:00,  1.01it/s]  


In [9]:
with open(f'{collection}_info.json', 'w') as f:
    json.dump(collection_info, f, indent=4)
with open(f'{collection}_nfts.json', 'w') as f:
    json.dump(nfts, f, indent=4)

In [10]:
with open(f'{collection}_info.json', 'r') as f:
    collection_info = json.load(f)
with open(f'{collection}_nfts.json', 'r') as f:
    nfts = json.load(f)

In [11]:
xoxno = {}

In [12]:
import scraper_utilities as su
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()
currency_path = '/html/body/div[1]/div[1]/main/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/div/div/div/img'
amount_path = '/html/body/div[1]/div[1]/main/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/div/div/div/span'
xoxno_address = 'erd1qqqqqqqqqqqqqpgq6wegs2xkypfpync8mn2sa5cmpqjlvrhwz5nqgepyg8'
for identifier, nft in tqdm(nfts.items()):
    if nft['owner'] == xoxno_address:
        try:
            url = f'https://xoxno.com/nft/{identifier}'
            driver.get(url)
            time.sleep(0.2)
            currency = driver.find_element(By.XPATH, currency_path).get_attribute('src').split('/')[-1].split('.')[0]
            amount = driver.find_element(By.XPATH, amount_path).text
            amount = float(amount.replace('K', ''))*1000 if 'K' in amount else float(amount)
            xoxno[identifier] = {
                'price': {
                    'currency': currency,
                    'amount': amount,
                }
            }
        except Exception as e:
            pass
driver.close()

100%|██████████| 2138/2138 [10:34<00:00,  3.37it/s]


In [13]:
with open(f'{collection}_xoxno.json', 'w') as f:
    json.dump(xoxno, f, indent=4)